In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [4]:
# Load the dataset
data = pd.read_csv('googleplaystore.csv')
data.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
# Data Cleaning
# Convert 'Last Updated' to datetime format
data['Last Updated'] = pd.to_datetime(data['Last Updated'], errors='coerce')

In [8]:
# Clean 'Installs' column by removing '+' and ',' then convert to integer
data['Installs'] = pd.to_numeric(data['Installs'].str.replace('[+,]', '', regex=True), errors='coerce')
data['Installs'] = data['Installs'].fillna(0).astype(int) # Replace NaN with 0 and convert to int

In [10]:
# Clean 'Size' column and convert to numeric (handling 'Varies with device')
data['Size'] = data['Size'].replace('Varies with device', pd.NA)
data['Size'] = data['Size'].str.replace('M', 'e6').str.replace('k', 'e3')
data['Size'] = pd.to_numeric(data['Size'], errors='coerce')

In [12]:
# Convert 'Reviews' to numeric
data['Reviews'] = pd.to_numeric(data['Reviews'], errors='coerce')

In [14]:
# Convert 'Rating' to numeric
data['Rating'] = pd.to_numeric(data['Rating'], errors='coerce')

In [16]:
# Function to check if the current time is within the allowed range
def is_time_allowed(start_hour, end_hour):
    current_hour = datetime.now().hour
    return start_hour <= current_hour < end_hour

## 3. Time Series Line Chart: Install Trends

In [19]:
if is_time_allowed(18, 21):
    filtered_data = data[
        (data['Content Rating'] == 'Teen') &
        (data['App'].str.startswith('E')) &
        (data['Installs'] > 10_000)
    ]

    if not filtered_data.empty:
        filtered_data['Month'] = filtered_data['Last Updated'].dt.to_period('M')
        installs_trend = filtered_data.groupby(['Month', 'Category'])['Installs'].sum().unstack()

        ax = installs_trend.plot(kind='line', figsize=(12, 6), marker='o')
        
        for category in installs_trend.columns:
            growth = installs_trend[category].pct_change() > 0.2
            ax.fill_between(
                installs_trend.index.to_timestamp(),
                installs_trend[category],
                where=growth,
                alpha=0.3
            )

        plt.title('Install Trends Over Time (Highlighted Growth > 20%)')
        plt.xlabel('Time')
        plt.ylabel('Total Installs')
        plt.legend(title='Category')
        plt.tight_layout()
        plt.show()
    else:
        print("No data available for the specified criteria.")
else:
    print("This graph is not available outside the allowed time (6 PM to 9 PM IST).")

This graph is not available outside the allowed time (6 PM to 9 PM IST).
